# Data analytics

## Setup

Requires Python 3.10 and Poetry 1.6. Run setup commands to begin:
```
!poetry install
!poetry run jupyter lab
```

In [1]:
import os
import copy
import math
import itertools
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

pd.options.display.max_columns = None
display_plots = False

## Load data

In [2]:
def get_raw_data():
    return {
        "A": {
            "test_est": pd.read_parquet(os.path.join("data", "A", "X_test_estimated.parquet")),
            "train_est": pd.read_parquet(os.path.join("data", "A", "X_train_estimated.parquet")),
            "train_obs": pd.read_parquet(os.path.join("data", "A", "X_train_observed.parquet")),
            "train_tar": pd.read_parquet(os.path.join("data", "A", "train_targets.parquet")),
        },
        "B": {
            "test_est": pd.read_parquet(os.path.join("data", "B", "X_test_estimated.parquet")),
            "train_est": pd.read_parquet(os.path.join("data", "B", "X_train_estimated.parquet")),
            "train_obs": pd.read_parquet(os.path.join("data", "B", "X_train_observed.parquet")),
            "train_tar": pd.read_parquet(os.path.join("data", "B", "train_targets.parquet")),
        },
        "C": {
            "test_est": pd.read_parquet(os.path.join("data", "C", "X_test_estimated.parquet")),
            "train_est": pd.read_parquet(os.path.join("data", "C", "X_train_estimated.parquet")),
            "train_obs": pd.read_parquet(os.path.join("data", "C", "X_train_observed.parquet")),
            "train_tar": pd.read_parquet(os.path.join("data", "C", "train_targets.parquet")),
        },
    }
data = get_raw_data()

View raw data

In [3]:
for loc in data:
    for type in data[loc]:
        display(loc, type, data[loc][type])

'A'

'test_est'

,date_calc,date_forecast,absolute_humidity_2m:gm3,air_density_2m:kgm3,ceiling_height_agl:m,clear_sky_energy_1h:J,clear_sky_rad:W,cloud_base_agl:m,dew_or_rime:idx,dew_point_2m:K,diffuse_rad:W,diffuse_rad_1h:J,direct_rad:W,direct_rad_1h:J,effective_cloud_cover:p,elevation:m,fresh_snow_12h:cm,fresh_snow_1h:cm,fresh_snow_24h:cm,fresh_snow_3h:cm,fresh_snow_6h:cm,is_day:idx,is_in_shadow:idx,msl_pressure:hPa,precip_5min:mm,precip_type_5min:idx,pressure_100m:hPa,pressure_50m:hPa,prob_rime:p,rain_water:kgm2,relative_humidity_1000hPa:p,sfc_pressure:hPa,snow_density:kgm3,snow_depth:cm,snow_drift:idx,snow_melt_10min:mm,snow_water:kgm2,sun_azimuth:d,sun_elevation:d,super_cooled_liquid_water:kgm2,t_1000hPa:K,total_cloud_cover:p,visibility:m,wind_speed_10m:ms,wind_speed_u_10m:ms,wind_speed_v_10m:ms,wind_speed_w_1000hPa:ms
0,2023-04-30 07:00:04,2023-05-01 00:00:00,4.4,1.286,912.700012,0.0,0.0,1041.199951,0.0,271.700012,0.0,0.0,0.0,0.0,80.699997,6.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1013.900024,0.0,0.0,1000.700012,1007.000000,0.0,0.0,80.300003,1013.299988,NaN,0.0,0.0,0.0,0.0,10.520000,-11.185,0.0,273.799988,80.699997,30210.699219,4.0,2.2,3.4,-0.0
1,2023-04-30 07:00:04,2023-05-01 00:15:00,4.3,1.287,912.700012,0.0,0.0,1054.800049,0.0,271.700012,0.0,0.0,0.0,0.0,77.000000,6.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1013.700012,0.0,0.0,1000.599976,1006.900024,0.0,0.0,80.300003,1013.200012,NaN,0.0,0.0,0.0,0.0,14.203000,-10.825,0.0,273.799988,77.000000,30003.599609,4.0,2.1,3.4,-0.0
2,2023-04-30 07:00:04,2023-05-01 00:30:00,4.3,1.287,912.700012,0.0,0.0,1068.300049,0.0,271.600006,0.0,0.0,0.0,0.0,73.099998,6.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1013.599976,0.0,0.0,1000.500000,1006.700012,0.0,0.0,80.300003,1013.000000,NaN,0.0,0.0,0.0,0.0,17.868999,-10.360,0.0,273.799988,73.099998,29797.099609,3.9,2.1,3.3,-0.0
3,2023-04-30 07:00:04,2023-05-01 00:45:00,4.3,1.287,912.700012,0.0,0.0,1081.900024,0.0,271.600006,0.0,0.0,0.0,0.0,69.000000,6.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1013.500000,0.0,0.0,1000.400024,1006.599976,0.0,0.0,80.199997,1012.900024,NaN,0.0,0.0,0.0,0.0,21.514000,-9.794,0.0,273.799988,69.000000,29618.599609,3.9,2.0,3.3,-0.0
4,2023-04-30 07:00:04,2023-05-01 01:00:00,4.3,1.287,NaN,0.0,0.0,1095.400024,0.0,271.600006,0.0,0.0,0.0,0.0,64.500000,6.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1013.299988,0.0,0.0,1000.200012,1006.500000,0.0,0.0,80.199997,1012.799988,NaN,0.0,0.0,0.0,0.0,25.135000,-9.128,0.0,273.799988,64.500000,29507.500000,3.9,2.0,3.3,-0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2875,2023-07-02 07:00:31,2023-07-03 22:45:00,9.1,1.207,2090.600098,0.0,0.0,1629.400024,0.0,282.700012,0.0,0.0,0.0,0.0,99.300003,6.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,993.299988,0.0,0.0,980.799988,986.700012,0.0,0.0,82.199997,992.599976,NaN,0.0,0.0,-0.0,0.0,351.855988,-3.282,0.1,285.399994,99.400002,39931.601562,2.1,1.7,-1.3,0.0
2876,2023-07-02 07:00:31,2023-07-03 23:00:00,9.1,1.207,2015.800049,0.0,0.0,1537.300049,0.0,282.700012,0.0,0.0,0.0,0.0,100.000000,6.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,993.400024,0.0,0.0,980.900024,986.799988,0.0,0.0,82.500000,992.700012,NaN,0.0,0.0,-0.0,0.0,355.317993,-3.469,0.1,285.299988,100.000000,39574.300781,2.0,1.6,-1.3,0.0
2877,2023-07-02 07:00:31,2023-07-03 23:15:00,9.1,1.208,2015.699951,0.0,0.0,1471.400024,0.0,282.700012,0.0,0.0,0.0,0.0,100.000000,6.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,993.500000,0.0,0.0,981.000000,986.900024,0.0,0.0,82.800003,992.799988,NaN,0.0,0.0,-0.0,0.0,358.785004,-3.556,0.1,285.299988,100.000000,39627.199219,2.0,1.6,-1.2,0.0
2878,2023-07-02 07:00:31,2023-07-03 23:30:00,9.0,1.208,2015.599976,0.0,0.0,1405.500000,0.0,282.700012,0.0,0.0,0.0,0.0,100.000000,6.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,993.599976,0.0,0.0,981.000000,986.900024,0.0,0.0,83.199997,992.900024,NaN,0.0,0.0,-0.0,0.0,2.253000,-3.541,0.1,285.200012,100.000000,39691.800781,2.0,1.6,-1.1,0.0


'A'

'train_est'

...

'train_obs'

,date_forecast,absolute_humidity_2m:gm3,air_density_2m:kgm3,ceiling_height_agl:m,clear_sky_energy_1h:J,clear_sky_rad:W,cloud_base_agl:m,dew_or_rime:idx,dew_point_2m:K,diffuse_rad:W,diffuse_rad_1h:J,direct_rad:W,direct_rad_1h:J,effective_cloud_cover:p,elevation:m,fresh_snow_12h:cm,fresh_snow_1h:cm,fresh_snow_24h:cm,fresh_snow_3h:cm,fresh_snow_6h:cm,is_day:idx,is_in_shadow:idx,msl_pressure:hPa,precip_5min:mm,precip_type_5min:idx,pressure_100m:hPa,pressure_50m:hPa,prob_rime:p,rain_water:kgm2,relative_humidity_1000hPa:p,sfc_pressure:hPa,snow_density:kgm3,snow_depth:cm,snow_drift:idx,snow_melt_10min:mm,snow_water:kgm2,sun_azimuth:d,sun_elevation:d,super_cooled_liquid_water:kgm2,t_1000hPa:K,total_cloud_cover:p,visibility:m,wind_speed_10m:ms,wind_speed_u_10m:ms,wind_speed_v_10m:ms,wind_speed_w_1000hPa:ms
0,2019-01-01 00:00:00,5.6,1.240,1153.599976,0.0,0.0,1153.599976,1.0,275.200012,0.0,0.0,0.0,0.0,63.599998,24.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,987.299988,0.00,0.0,972.599976,978.599976,0.0,0.0,90.199997,984.700012,NaN,0.0,0.0,-0.0,0.0,12.989000,-49.178001,0.0,278.500000,68.500000,18220.400391,2.5,1.1,2.2,-0.0
1,2019-01-01 00:15:00,5.5,1.241,1137.900024,0.0,0.0,1137.900024,1.0,275.100006,0.0,0.0,0.0,0.0,74.900002,24.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,987.099976,0.06,1.0,972.299988,978.299988,0.2,0.0,90.000000,984.400024,NaN,0.0,0.0,-0.0,0.0,18.214001,-48.724998,0.0,278.399994,80.800003,17083.699219,2.8,1.1,2.5,-0.0
2,2019-01-01 00:30:00,5.4,1.241,1122.199951,0.0,0.0,1122.199951,1.0,274.899994,0.0,0.0,0.0,0.0,84.300003,24.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,986.799988,0.06,1.0,972.000000,978.000000,0.3,0.0,89.900002,984.099976,NaN,0.0,0.0,-0.0,0.1,23.351000,-48.127998,0.0,278.299988,89.900002,15940.900391,3.0,1.1,2.8,-0.0
3,2019-01-01 00:45:00,5.4,1.241,1106.400024,0.0,0.0,1106.400024,1.0,274.700012,0.0,0.0,0.0,0.0,92.000000,24.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,986.599976,0.06,1.0,971.700012,977.799988,0.3,0.0,89.699997,983.799988,NaN,0.0,0.0,-0.0,0.1,28.382000,-47.393002,0.0,278.200012,96.099998,14792.000000,3.3,1.2,3.0,-0.0
4,2019-01-01 01:00:00,5.3,1.241,1090.599976,0.0,0.0,1090.599976,1.0,274.500000,0.0,0.0,0.0,0.0,98.000000,24.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,986.400024,0.06,1.0,971.400024,977.500000,0.4,0.0,89.599998,983.599976,NaN,0.0,0.0,-0.0,0.1,33.292999,-46.529999,0.0,278.100006,99.400002,13637.299805,3.5,1.2,3.3,-0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
116820,2022-05-01 21:00:00,4.9,1.257,2652.000000,0.0,0.0,1536.500000,0.0,273.600006,0.0,0.0,0.0,0.0,95.000000,24.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1014.799988,0.00,0.0,999.299988,1005.400024,0.0,0.0,62.599998,1011.599976,NaN,0.0,0.0,-0.0,0.0,327.303009,-7.367000,0.0,279.299988,100.000000,38312.000000,4.7,3.0,3.6,-0.0
116821,2022-05-01 21:15:00,4.9,1.256,2375.300049,0.0,0.0,1484.500000,0.0,273.600006,0.0,0.0,0.0,0.0,97.000000,24.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1014.500000,0.00,0.0,999.000000,1005.099976,0.0,0.0,62.799999,1011.299988,NaN,0.0,0.0,-0.0,0.0,330.863007,-8.232000,0.1,279.299988,100.000000,38344.199219,4.8,3.2,3.6,-0.0
116822,2022-05-01 21:30:00,4.9,1.256,2098.699951,0.0,0.0,1432.500000,0.0,273.500000,0.0,0.0,0.0,0.0,98.500000,24.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1014.200012,0.00,0.0,998.599976,1004.799988,0.0,0.0,63.000000,1010.900024,NaN,0.0,0.0,-0.0,0.0,334.451996,-9.006000,0.1,279.200012,100.000000,38402.601562,4.9,3.3,3.6,-0.0
116823,2022-05-01 21:45:00,4.8,1.255,1822.000000,0.0,0.0,1380.400024,0.0,273.399994,0.0,0.0,0.0,0.0,99.500000,24.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1013.900024,0.00,0.0,998.299988,1004.400024,0.0,0.0,63.200001,1010.599976,NaN,0.0,0.0,-0.0,0.0,338.067993,-9.683000,0.1,279.200012,100.000000,38478.800781,5.0,3.5,3.6,-0.0


'C'

'train_tar'

,time,pv_measurement
0,2018-12-31 23:00:00,NaN
1,2019-01-01 00:00:00,NaN
2,2019-01-01 01:00:00,NaN
3,2019-01-01 02:00:00,NaN
4,2019-01-01 03:00:00,NaN
...,...,...
32150,2023-04-30 19:00:00,50.96
32151,2023-04-30 20:00:00,2.94
32152,2023-04-30 21:00:00,0.00
32153,2023-04-30 22:00:00,-0.00


Find the number of unique values for each type

In [4]:
pd.concat([data[loc][type] for loc in data for type in data[loc]]).nunique()

date_calc                            214
date_forecast                     153805
absolute_humidity_2m:gm3             183
air_density_2m:kgm3                  301
ceiling_height_agl:m               78367
clear_sky_energy_1h:J             217030
clear_sky_rad:W                     8371
cloud_base_agl:m                   61031
dew_or_rime:idx                        3
dew_point_2m:K                       473
diffuse_rad:W                       3204
diffuse_rad_1h:J                  215787
direct_rad:W                        6590
direct_rad_1h:J                   180598
effective_cloud_cover:p             1001
elevation:m                            3
fresh_snow_12h:cm                    184
fresh_snow_1h:cm                      58
fresh_snow_24h:cm                    205
fresh_snow_3h:cm                     104
fresh_snow_6h:cm                     145
is_day:idx                             2
is_in_shadow:idx                       2
msl_pressure:hPa                     981
precip_5min:mm  

Plot target value

In [5]:
if display_plots:
    for loc in data:
        obs_start = data[loc]["train_obs"]["date_forecast"].min()
        obs_end = data[loc]["train_obs"]["date_forecast"].max()
        est_start = data[loc]["train_est"]["date_forecast"].min()
        est_end = data[loc]["train_est"]["date_forecast"].max()
        
        ax = data[loc]["train_tar"].set_index("time").plot(lw=0.1)
        ax.axvline(obs_start, linestyle='--')
        ax.axvline(obs_end, linestyle='--')
        ax.axvline(est_start, linestyle='--')
        ax.axvline(est_end, linestyle='--')
        ax.set_title("pv_measurement")
        plt.show()

## Transform data

Transform the raw data to be on the X y form

In [6]:
def transform_data(data):
    # Create empty list of location train data and location test data
    Xs = {loc : {} for loc in data}
    ys = {loc : {} for loc in data}
    
    for loc in data:
        # Copy data
        data_obs = data[loc]["train_obs"].copy()
        data_est = data[loc]["train_est"].copy()
        data_test = data[loc]["test_est"].copy()
        data_tar = data[loc]["train_tar"].copy()
        
        # Add a column with the difference between date_forecast and date_calc in seconds and 
        data_obs["forecast_seconds"] = 0
        data_est["forecast_seconds"] = (data_est["date_forecast"] - data_est["date_calc"]).apply(lambda x: x.total_seconds())
        data_test["forecast_seconds"] = (data_test["date_forecast"] - data_test["date_calc"]).apply(lambda x: x.total_seconds())
    
        # Remove unneeded columns
        del data_est["date_calc"]
        del data_test["date_calc"]
        del data_obs["snow_density:kgm3"]
        del data_est["snow_density:kgm3"]
        del data_test["snow_density:kgm3"]

        # Find start and end date for obvsersed and estimated data
        obs_end = data_obs["date_forecast"].max()
        est_start = data_est["date_forecast"].min()
        treshold_date = obs_end + (est_start - obs_end) / 2
        
        # Outer join target data with estimated and observed data
        Xy_obs = pd.merge(
            data_tar[data_tar["time"] <= treshold_date],
            data_obs.rename(columns={"date_forecast": "time"}),
            on="time",
            how="outer"
        ).set_index("time").sort_index()
        Xy_est = pd.merge(
            data_tar[data_tar["time"] > treshold_date],
            data_est.rename(columns={"date_forecast": "time"}),
            on="time",
            how="outer"
        ).set_index("time").sort_index()

        # Assign Xs
        Xs[loc]["obs"] = Xy_obs.drop("pv_measurement", axis=1)
        Xs[loc]["est"] = Xy_est.drop("pv_measurement", axis=1)
        Xs[loc]["test"] = data_test.rename(columns={"date_forecast": "time"}).set_index("time").sort_index()

        # Assign ys
        ys[loc]["obs"] = Xy_obs["pv_measurement"]
        ys[loc]["est"] = Xy_est["pv_measurement"]
        
    return Xs, ys

Xs, ys = transform_data(data)

Check that they have the same columns

In [7]:
def has_same_columns(df_lst):
    return all(set(df.columns) == set(df_lst[0].columns) for df in df_lst)

def are_all_equal(lst):
    return all(x == lst[0] for x in lst)

test_dates = {loc: Xs[loc]["test"].index.copy() for loc in Xs}
Xs_cols = Xs["A"]["obs"].columns.copy()
ys_col = ys["A"]["obs"].name

display(
    has_same_columns([Xs[loc][type] for loc in Xs for type in Xs[loc]]), 
    are_all_equal([ys[loc][type].name for loc in ys for type in ys[loc]])
)

True

True

See stats about the data

In [8]:
pd.concat([Xs[loc][type] for loc in Xs for type in Xs[loc]]).describe()

,absolute_humidity_2m:gm3,air_density_2m:kgm3,ceiling_height_agl:m,clear_sky_energy_1h:J,clear_sky_rad:W,cloud_base_agl:m,dew_or_rime:idx,dew_point_2m:K,diffuse_rad:W,diffuse_rad_1h:J,direct_rad:W,direct_rad_1h:J,effective_cloud_cover:p,elevation:m,fresh_snow_12h:cm,fresh_snow_1h:cm,fresh_snow_24h:cm,fresh_snow_3h:cm,fresh_snow_6h:cm,is_day:idx,is_in_shadow:idx,msl_pressure:hPa,precip_5min:mm,precip_type_5min:idx,pressure_100m:hPa,pressure_50m:hPa,prob_rime:p,rain_water:kgm2,relative_humidity_1000hPa:p,sfc_pressure:hPa,snow_depth:cm,snow_drift:idx,snow_melt_10min:mm,snow_water:kgm2,sun_azimuth:d,sun_elevation:d,super_cooled_liquid_water:kgm2,t_1000hPa:K,total_cloud_cover:p,visibility:m,wind_speed_10m:ms,wind_speed_u_10m:ms,wind_speed_v_10m:ms,wind_speed_w_1000hPa:ms,forecast_seconds
count,413791.000000,413791.000000,338248.000000,4.137910e+05,413791.000000,381929.000000,413791.000000,413791.000000,413791.000000,4.137910e+05,413791.000000,4.137910e+05,413791.000000,413791.000000,413791.000000,413791.000000,413791.000000,413791.000000,413791.000000,413791.000000,413791.000000,413791.000000,413791.000000,413791.000000,413791.000000,413791.000000,413791.000000,413791.000000,413791.000000,413791.000000,413791.000000,413791.000000,413791.000000,413791.000000,413791.000000,413791.000000,413791.000000,413791.000000,413791.000000,413791.000000,413791.000000,413791.000000,413791.000000,413791.000000,413791.000000
mean,6.022480,1.255468,2796.203857,5.340971e+05,148.360306,1717.518188,0.002364,275.193665,40.693062,1.464950e+05,51.762341,1.863446e+05,67.066154,12.303769,0.139911,0.011645,0.276628,0.035001,0.069955,0.490383,0.559183,1009.625610,0.005670,0.084200,995.809204,1001.939941,0.851963,0.009438,73.572304,1008.098511,0.322603,0.000027,0.000382,0.088049,179.594162,-0.685629,0.056825,279.437927,73.616234,33047.644531,3.017195,0.677667,0.670873,0.000048,15049.953856
std,2.767936,0.037358,2488.790527,8.383486e+05,233.465897,1775.247437,0.253579,7.009235,61.931904,2.201175e+05,115.628448,4.105117e+05,34.583103,8.251439,0.842233,0.118890,1.340529,0.297705,0.511651,0.499908,0.496486,12.998659,0.032662,0.386233,12.919853,12.974976,5.750685,0.042001,14.360833,13.032352,1.823088,0.005156,0.005010,0.238048,103.120628,24.189608,0.108665,6.691157,34.396370,17905.146484,1.749300,2.778564,1.879975,0.006740,37329.289934
min,0.500000,1.139000,27.600000,0.000000e+00,0.000000,27.299999,-1.000000,247.300003,0.000000,0.000000e+00,0.000000,0.000000e+00,0.000000,6.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,944.299988,0.000000,0.000000,929.799988,935.599976,0.000000,0.000000,18.700001,941.400024,0.000000,0.000000,-0.000000,0.000000,0.002000,-50.042000,0.000000,257.899994,0.000000,130.600006,0.000000,-7.300000,-9.300000,-0.100000,0.000000
25%,4.000000,1.230000,1048.199951,0.000000e+00,0.000000,596.700012,0.000000,270.600006,0.000000,0.000000e+00,0.000000,0.000000e+00,41.599998,6.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1001.500000,0.000000,0.000000,987.799988,993.900024,0.000000,0.000000,64.099998,1000.000000,0.000000,0.000000,0.000000,0.000000,89.743500,-17.895000,0.000000,274.799988,52.400002,16864.900391,1.700000,-1.300000,-0.600000,-0.000000,0.000000
50%,5.400000,1.255000,1821.800049,9.416100e+03,0.000000,1158.300049,0.000000,275.000000,0.000000,1.198950e+04,0.000000,0.000000e+00,80.400002,7.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1010.500000,0.000000,0.000000,996.700012,1002.799988,0.000000,0.000000,76.000000,1009.000000,0.000000,0.000000,0.000000,0.000000,179.462997,-0.563000,0.000000,278.700012,93.699997,37022.199219,2.700000,0.300000,0.700000,0.000000,0.000000
75%,7.900000,1.279000,3795.424805,8.397916e+05,233.699997,2067.500000,0.000000,280.600006,68.300003,2.453228e+05,30.500000,1.181203e+05,98.900002,24.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,1018.599976,0.000000,0.000000,1004.900024,1011.000000,0.000000,0.000000,85.099998,1017.20001

## Visualize data

Plot timeperiod of test dates

In [9]:
if display_plots:
    for height, loc in zip([3,2,1], test_dates):
        plt.plot(test_dates[loc], [height] * len(test_dates[loc]), marker='o', markersize=1, linestyle='')
    plt.ylim(0, 4)
    plt.legend([loc for loc in test_dates])
    plt.gca().axes.get_yaxis().set_visible(False)
    plt.xticks(rotation=45)
    plt.show()

Display ys

In [10]:
if display_plots:
    for loc in ys:
        for type in ys[loc]:
            ax = ys[loc][type].dropna().plot(lw=0.1)
    
    ax.set_title(ys_col)
    legend = ax.legend([loc + " " + type for loc in ys for type in ys[loc]])
    for line in legend.get_lines():
        line.set_linewidth(1)
    plt.show()
    
    for loc in ys:
        for type in ys[loc]:
            ax = ys[loc][type].dropna().plot(lw=0.1)
    
        ax.set_title(loc)
        plt.show()

Display Xs only with nan values

In [11]:
Xs_cols_with_nan = Xs_cols[pd.concat([Xs[loc][type].dropna(how='all') for loc in Xs for type in Xs[loc]]).isna().any()]
display(Xs_cols_with_nan)

if display_plots:
    for col in Xs_cols_with_nan:
        for loc in Xs:
            for type in Xs[loc]:
                ax = Xs[loc][type][col].dropna().plot(lw=0.1)
    
        ax.set_title(col)
        legend = ax.legend([loc + " " + type for loc in ys for type in Xs[loc]])
        for line in legend.get_lines():
            line.set_linewidth(1)
        plt.show()

        for loc in Xs:
            for type in Xs[loc]:
                ax = Xs[loc][type][col].dropna().plot(lw=0.1)
        
            ax.set_title(loc)
            plt.show()

Index(['ceiling_height_agl:m', 'cloud_base_agl:m'], dtype='object')

Plot histogram of the data

In [12]:
if display_plots:
    pd.concat([Xs[loc][type] for loc in Xs for type in Xs[loc]]).hist(figsize=(20,20))
    plt.show()

Plot correlation matrix of the data

In [13]:
if display_plots:
    display(list(zip(range(len(Xs_cols)), Xs_cols)))
    plt.matshow(pd.concat([Xs[loc][type] for loc in Xs for type in Xs[loc]]).corr(numeric_only=True), cmap="PRGn", interpolation="none", vmin=-1, vmax=1)
    plt.show()

## Gap detection

In [14]:
def detect_timeseries_gaps(x, min_gap_time, time_padding="1 sec"):
    time_padding = pd.to_timedelta(time_padding)
    
    start_date = x.index[0] - time_padding
    start_val = x.iloc[0]

    gaps_date = []
    index = x.dropna().index
    
    for date in index:
        val = x[date]
        
        if(val != start_val):
            if(date - start_date > min_gap_time):
                gaps_date.append((start_date, date))
    
            start_date = date
            start_val = val

    i = min(x.index.get_loc(index[-1]) + 1, len(x) - 1)
    if(x.index[i] - start_date > min_gap_time):
        gaps_date.append((start_date, x.index[i]))

    if(x.index[-1] - x.index[i-1] > min_gap_time):
        gaps_date.append((x.index[i-1], x.index[-1] + time_padding))

    return gaps_date


ys_gaps = {loc : {} for loc in ys}
for loc in ys:
    for type in ys[loc]:
        ys_gaps[loc][type] = detect_timeseries_gaps(ys[loc][type], pd.Timedelta(days=1))
        display(loc, type, ys_gaps[loc][type])

'A'

'obs'

[(Timestamp('2020-01-04 14:00:00'), Timestamp('2020-01-06 09:00:00'))]

'A'

'est'

...

'obs'

[(Timestamp('2018-12-31 22:59:59'), Timestamp('2019-09-04 08:00:00')),
 (Timestamp('2019-09-04 09:00:00'), Timestamp('2019-09-05 13:00:00')),
 (Timestamp('2019-11-11 11:00:00'), Timestamp('2019-11-13 09:00:00')),
 (Timestamp('2019-11-28 14:00:00'), Timestamp('2019-12-05 10:00:00')),
 (Timestamp('2019-12-07 13:00:00'), Timestamp('2019-12-13 10:00:00')),
 (Timestamp('2019-12-16 13:00:00'), Timestamp('2019-12-21 10:00:00')),
 (Timestamp('2019-12-25 12:00:00'), Timestamp('2019-12-30 10:00:00')),
 (Timestamp('2020-01-02 13:00:00'), Timestamp('2020-01-07 10:00:00')),
 (Timestamp('2020-01-23 14:00:00'), Timestamp('2020-01-26 09:00:00')),
 (Timestamp('2020-02-05 13:00:00'), Timestamp('2020-02-10 08:00:00')),
 (Timestamp('2020-02-23 16:00:00'), Timestamp('2020-03-08 09:00:00')),
 (Timestamp('2020-03-28 17:00:00'), Timestamp('2020-03-31 10:00:00')),
 (Timestamp('2020-11-18 12:00:00'), Timestamp('2020-11-22 09:00:00')),
 (Timestamp('2020-12-16 13:00:00'), Timestamp('2020-12-18 09:00:00')),
 (Time

'C'

'est'

[(Timestamp('2022-12-05 13:00:00'), Timestamp('2023-01-08 12:00:00')),
 (Timestamp('2023-01-15 14:00:00'), Timestamp('2023-01-18 13:00:00')),
 (Timestamp('2023-02-01 14:00:00'), Timestamp('2023-02-04 10:00:00')),
 (Timestamp('2023-02-04 10:00:00'), Timestamp('2023-02-07 11:00:00')),
 (Timestamp('2023-02-09 15:00:00'), Timestamp('2023-02-12 09:00:00')),
 (Timestamp('2023-02-19 13:00:00'), Timestamp('2023-02-21 11:00:00')),
 (Timestamp('2023-02-21 15:00:00'), Timestamp('2023-02-23 10:00:00')),
 (Timestamp('2023-02-23 14:00:00'), Timestamp('2023-02-26 10:00:00')),
 (Timestamp('2023-03-05 11:00:00'), Timestamp('2023-03-18 08:00:00'))]

Visualize gaps

In [15]:
if display_plots:
    for loc in ys:
        for type in ys[loc]:
            ax = ys[loc][type].dropna().plot(lw=0.1)
            
            for start_date, end_date in ys_gaps[loc][type]:
                ax.axvline(start_date, linestyle='--', color="green")
                ax.axvline(end_date, linestyle='--', color="red")
            ax.set_title(loc)
        plt.show()

## Timeseries data preprocessing

Reindex to regular time series with 15 min interval

In [16]:
def reindex_timeseries(X, freq, start=None, end=None):
    if start is None:
        start = X.index[0]

    if end is None:
        end = X.index[-1]
    
    return X.reindex(pd.date_range(start=start, end=end, freq=freq))

for loc in Xs:
    for type in Xs[loc]:        
        Xs[loc][type] = reindex_timeseries(Xs[loc][type], freq = "15T")

for loc in ys:
    for type in ys[loc]:      
        ys[loc][type] = reindex_timeseries(ys[loc][type], freq = "15T")

Combine two timeseries into one

In [17]:
def cut_gap_timeseries(x, ts_gaps_dates):
    ts_copy = x.copy()
    
    for (start_date, end_date) in ts_gaps_dates:        
        gap_dates = x.index[(start_date < x.index) & (x.index < end_date)]
        ts_copy[gap_dates] = np.nan

    return ts_copy

def combine_timeseries(X1, X2):
    return (X1.combine_first(X2) + X2.combine_first(X1)) / 2

Xs["BC"] = {}
ys["BC"] = {}

for type in Xs["A"]:
    start = min(Xs["B"][type].index[0], Xs["C"][type].index[0])
    end = max(Xs["B"][type].index[-1], Xs["C"][type].index[-1])
    
    Xs["BC"][type] = combine_timeseries(
        reindex_timeseries(Xs["B"][type], freq = "15T", start=start, end=end),
        reindex_timeseries(Xs["C"][type], freq = "15T", start=start, end=end),
    )

for type in ys["A"]:
    start = min(ys["B"][type].index[0], ys["C"][type].index[0])
    end = max(ys["B"][type].index[-1], ys["C"][type].index[-1])
    
    ys["BC"][type] = combine_timeseries(
        reindex_timeseries(cut_gap_timeseries(ys["B"][type], ys_gaps["B"][type]), freq = "15T", start=start, end=end),
        reindex_timeseries(cut_gap_timeseries(ys["C"][type], ys_gaps["C"][type]), freq = "15T", start=start, end=end),
    )

Fill gaps in one timeseries with values from another timeseries

In [18]:
def fill_gaps_timeseries(x1, x2, x1_gaps_dates, x2_gaps_dates):
    common_dates = x1.index.intersection(x2.index)
    
    x1_copy = x1.copy()
    x2_copy = x2.copy()
    
    for (start_date, end_date) in x1_gaps_dates:        
        gap_dates = common_dates[(start_date < common_dates) & (common_dates < end_date)]
        x1_copy[gap_dates] = x2[gap_dates]

    for (start_date, end_date) in x2_gaps_dates:
        gap_dates = common_dates[(start_date < common_dates) & (common_dates < end_date)]
        x2_copy[gap_dates] = x1[gap_dates]

    return x1_copy, x2_copy

for type in ys["A"]:
    ys["B"][type], ys["C"][type] = fill_gaps_timeseries(
        ys["B"][type],
        ys["C"][type],
        ys_gaps["B"][type],
        ys_gaps["C"][type],
    )

Time interpolate missing values

In [19]:
def interpolate_timeseries(X):
    return X.interpolate(method='time', limit_direction="both")

for loc in Xs:
    for type in Xs[loc]:    
        Xs[loc][type] = interpolate_timeseries(Xs[loc][type])

for loc in ys:
    for type in ys[loc]:    
        ys[loc][type] = interpolate_timeseries(ys[loc][type])

## Validate timeseries preprocessing

Check for nan values

In [20]:
display("Xs", pd.concat([Xs[loc][type] for loc in Xs for type in Xs[loc]]).isna().sum())
display("ys", pd.concat([ys[loc][type] for loc in ys for type in ys[loc]]).isna().sum())

'Xs'

absolute_humidity_2m:gm3          0
air_density_2m:kgm3               0
ceiling_height_agl:m              0
clear_sky_energy_1h:J             0
clear_sky_rad:W                   0
cloud_base_agl:m                  0
dew_or_rime:idx                   0
dew_point_2m:K                    0
diffuse_rad:W                     0
diffuse_rad_1h:J                  0
direct_rad:W                      0
direct_rad_1h:J                   0
effective_cloud_cover:p           0
elevation:m                       0
fresh_snow_12h:cm                 0
fresh_snow_1h:cm                  0
fresh_snow_24h:cm                 0
fresh_snow_3h:cm                  0
fresh_snow_6h:cm                  0
is_day:idx                        0
is_in_shadow:idx                  0
msl_pressure:hPa                  0
precip_5min:mm                    0
precip_type_5min:idx              0
pressure_100m:hPa                 0
pressure_50m:hPa                  0
prob_rime:p                       0
rain_water:kgm2             

'ys'

0

Check that the period is correct

In [21]:
for loc in Xs:
    for type in Xs[loc]:
        display("", "Xs " + loc + " " + type, Xs[loc][type].index[0], Xs[loc][type].index[-1], Xs[loc][type].index.to_series().diff().value_counts())

for loc in ys:
    for type in ys[loc]:
        display("", "ys " + loc + " " + type, ys[loc][type].index[0], ys[loc][type].index[-1], ys[loc][type].index.to_series().diff().value_counts())

''

'Xs A obs'

Timestamp('2019-06-02 22:00:00')

Timestamp('2022-10-21 01:00:00')

0 days 00:15:00    118668
Name: count, dtype: int64

...

''

'ys BC est'

Timestamp('2022-10-28 22:00:00')

Timestamp('2023-04-30 23:45:00')

0 days 00:15:00    17671
Name: count, dtype: int64

Display ys

In [22]:
if display_plots:
    for loc in ys:
        for type in ys[loc]:
            ax = ys[loc][type].dropna().plot(lw=0.1)
        ax.set_title(name + " " + loc)
        plt.show()

## More data preprocessing

One hot encoding of categorial values in X

In [23]:
def one_hot_encoding(X):
    return pd.get_dummies(X)

Z-score normalizing of numeric values in X

In [24]:
class StandardScaler:
    def __init__(self, X):
        self.means = X.mean(numeric_only=True)
        self.stds = X.std(numeric_only=True)

    def normalize(self, X):
        indices = self.indices(X)
        X_copy = X.copy()
        X_copy[indices] = (X[indices] - self.means[indices]) / self.stds[indices]
        return X_copy

    def denormalize(self, X):
        indices = self.indices(X)
        X_copy = X.copy()
        X_copy[indices] = X[indices] * self.stds[indices] + self.means[indices]
        return X_copy

    def indices(self, X):
        return self.means.index.intersection(X.columns)

Polynomial expansion of numeric values in X to degree n
    
Example:
If [x y] = X then
- degree 1: [1 x y]
- degree 2: [1 x y xx xy yy]
- degree 3: [1 x y xx xy yy xxx xxy xyy yyy]
- ...

Note: Since multiplication is commutitative then 
    xy = yx, xxy = xyx = yxx, xyy = yxy = yyx
so we skip non-lexicographically sorted multiplications.

In [25]:
def is_sorted(lst):
    return all(lst[i] <= lst[i+1] for i in range(len(lst) - 1))

def polynomial_expansion(X, degree=2, with_constant_term=False):
    cols = X.select_dtypes(include=["number"]).columns
    X_num = X.select_dtypes(include=["number"]).to_numpy()
    (m,n) = X_num.shape

    xs = [X]
    if(with_constant_term):
        xs.append(pd.Series(np.ones(m), name='1'))

    for d in range(2, degree+1):
        for indices in filter(is_sorted, itertools.product(range(n), repeat=d)):
            x = np.ones(m)
            for j in indices:
                x *= X_num[:,j]
            xs.append(pd.Series(x, name="_x_".join([cols[i] for i in indices])))

    return pd.concat(xs, axis=1)

Test functions

In [26]:
X = pd.DataFrame({
    'A': [1, 2, 3],
    'B': [4, 5, 6],
    'C': [7, 8, 9],
    'D': ['a', 'b', 'c'],
})

scaler = StandardScaler(X)
X_norm = scaler.normalize(X)

print(scaler.denormalize(X_norm))
print(X_norm)
print(one_hot_encoding(X))
print(polynomial_expansion(X))

     A    B    C  D
0  1.0  4.0  7.0  a
1  2.0  5.0  8.0  b
2  3.0  6.0  9.0  c
     A    B    C  D
0 -1.0 -1.0 -1.0  a
1  0.0  0.0  0.0  b
2  1.0  1.0  1.0  c
   A  B  C    D_a    D_b    D_c
0  1  4  7   True  False  False
1  2  5  8  False   True  False
2  3  6  9  False  False   True
   A  B  C  D  A_x_A  A_x_B  A_x_C  B_x_B  B_x_C  C_x_C
0  1  4  7  a    1.0    4.0    7.0   16.0   28.0   49.0
1  2  5  8  b    4.0   10.0   16.0   25.0   40.0   64.0
2  3  6  9  c    9.0   18.0   27.0   36.0   54.0   81.0


Normalize Xs and ys to standard scalar with sklearn

In [27]:
from sklearn.preprocessing import StandardScaler

X = pd.concat([Xs[loc][type] for loc in Xs for type in Xs[loc]])
y = pd.concat([ys[loc][type] for loc in ys for type in ys[loc]])

scaler_X = StandardScaler().fit(X.to_numpy())
scaler_y = StandardScaler().fit(y.to_numpy().reshape(-1, 1))

Xs_norm = {loc : {} for loc in Xs}
for loc in Xs:
    for type in Xs[loc]:
        Xs_norm[loc][type] = pd.DataFrame(
            scaler_X.transform(Xs[loc][type].to_numpy()), 
            columns = Xs[loc][type].columns, 
            index = Xs[loc][type].index,
        )
        display(loc, type, Xs_norm[loc][type])

ys_norm = {loc : {} for loc in ys}
for loc in ys:
    for type in ys[loc]:
        ys_norm[loc][type] = pd.Series(
            scaler_y.transform(ys[loc][type].to_numpy().reshape(-1, 1)).flatten(), 
            name = ys[loc][type].name, 
            index = ys[loc][type].index,
        )
        display(loc, type, ys_norm[loc][type])

'A'

'obs'

,absolute_humidity_2m:gm3,air_density_2m:kgm3,ceiling_height_agl:m,clear_sky_energy_1h:J,clear_sky_rad:W,cloud_base_agl:m,dew_or_rime:idx,dew_point_2m:K,diffuse_rad:W,diffuse_rad_1h:J,direct_rad:W,direct_rad_1h:J,effective_cloud_cover:p,elevation:m,fresh_snow_12h:cm,fresh_snow_1h:cm,fresh_snow_24h:cm,fresh_snow_3h:cm,fresh_snow_6h:cm,is_day:idx,is_in_shadow:idx,msl_pressure:hPa,precip_5min:mm,precip_type_5min:idx,pressure_100m:hPa,pressure_50m:hPa,prob_rime:p,rain_water:kgm2,relative_humidity_1000hPa:p,sfc_pressure:hPa,snow_depth:cm,snow_drift:idx,snow_melt_10min:mm,snow_water:kgm2,sun_azimuth:d,sun_elevation:d,super_cooled_liquid_water:kgm2,t_1000hPa:K,total_cloud_cover:p,visibility:m,wind_speed_10m:ms,wind_speed_u_10m:ms,wind_speed_v_10m:ms,wind_speed_w_1000hPa:ms,forecast_seconds
2019-06-02 22:00:00,0.612617,-0.695508,-0.519220,-0.623954,-0.622388,-0.076851,-0.014095,0.734235,-0.643929,-0.652121,-0.439971,-0.446065,0.924236,-0.974891,-0.173847,-0.103819,-0.214494,-0.124194,-0.14371,-0.954833,0.867405,-0.233638,-0.176078,-0.218363,-0.131859,-0.133820,-0.168573,-0.223429,-0.032479,-0.145523,-0.183148,-0.005623,-0.080911,0.056768,1.591467,-0.093442,-0.525923,0.969646,0.776206,0.373819,0.408989,-1.560864,-0.798687,-0.007821,-0.435066
2019-06-02 22:15:00,0.612617,-0.722430,-0.523227,-0.623954,-0.622388,-0.082433,-0.014095,0.734235,-0.643929,-0.652121,-0.439971,-0.446065,0.932892,-0.974891,-0.173847,-0.103819,-0.214494,-0.124194,-0.14371,-0.954833,0.867405,-0.256725,-0.176078,-0.218363,-0.155108,-0.164684,-0.168573,-0.223429,-0.095389,-0.168570,-0.183148,-0.005623,-0.080911,0.481780,1.624377,-0.112162,-0.525923,0.999635,0.776206,0.401070,0.351447,-1.560864,-0.691341,-0.007821,-0.435066
2019-06-02 22:30:00,0.612617,-0.749349,-0.527086,-0.623954,-0.622388,-0.087809,-0.014095,0.734235,-0.643929,-0.652121,-0.439971,-0.446065,0.938662,-0.974891,-0.173847,-0.103819,-0.214494,-0.124194,-0.14371,-0.954833,0.867405,-0.287511,-0.176078,-0.218363,-0.178361,-0.187839,-0.168573,-0.223429,-0.165289,-0.199303,-0.183148,-0.005623,-0.080911,0.481780,1.657419,-0.126703,-0.525923,1.029619,0.776206,0.429481,0.351447,-1.560864,-0.583996,-0.007821,-0.435066
2019-06-02 22:45:00,0.612617,-0.803193,-0.530799,-0.623954,-0.622388,-0.092981,-0.014095,0.734235,-0.643929,-0.652121,-0.439971,-0.446065,0.944432,-0.974891,-0.173847,-0.103819,-0.214494,-0.124194,-0.14371,-0.954833,0.867405,-0.310598,-0.176078,-0.218363,-0.201610,-0.210988,-0.168573,-0.223429,-0.235189,-0.222350,-0.183148,-0.005623,-0.080911,0.481780,1.690548,-0.137108,-0.525923,1.074602,0.776206,0.459069,0.293906,-1.524372,-0.476651,-0.007821,-0.435066
2019-06-02 23:00:00,0.612617,-0.830112,-0.534401,-0.623954,-0.622388,-0.097999,-0.014095,0.734235,-0.643929,-0.652121,-0.439971,-0.446065,0.950203,-0.974891,-0.173847,-0.103819,-0.214494,-0.124194,-0.14371,-0.954833,0.867405,-0.333685,-0.176078,-0.218363,-0.224859,-0.234138,-0.168573,-0.223429,-0.305089,-0.245397,-0.183148,-0.005623,-0.080911,0.481780,1.723752,-0.143209,-0.525923,1.104587,0.776206,0.489841,0.293906,-1.524372,-0.369305,-0.007821,-0.435066
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-10-21 00:00:00,-0.366921,0.973592,-0.685885,-0.623954,-0.622388,-0.893764,-0.014095,-0.210112,-0.643929,-0.652121,-0.439971,-0.446065,-1.842610,-0.974891,-0.173847,-0.103819,-0.214494,-0.124194,-0.14371,-0.954833,0.867405,0.659097,-0.176078,-0.218363,0.743876,0.753618,-0.168573,-0.223429,0.638560,0.761036,-0.183148,-0.005623,-0.080911,-0.368244,-1.511361,-1.476712,-0.525923,-0.379768,-2.047364,-0.406257,-0.972010,-0.393108,0.274767,-0.007821,-0.435066
2022-10-21 00:15:00,-0.366921,0.973592,-0.685885,-0.623954,-0.622388,-0.893764,-0.014095,-0.210112,-0.643929,-0.652121,-0.439971,-0.446065,-1.857036,-0.974891,-0.173847,-0.103819,-0.214494,-0.124194,-0.14371,-0.954833,0.867405,0.651398,-0.176078,-0.218363,0.736128,0

'A'

'est'

...

'obs'

2018-12-31 23:00:00   -0.354993
2018-12-31 23:15:00   -0.354993
2018-12-31 23:30:00   -0.354993
2018-12-31 23:45:00   -0.354993
2019-01-01 00:00:00   -0.354993
                         ...   
2022-05-02 23:00:00   -0.354993
2022-05-02 23:15:00   -0.354993
2022-05-02 23:30:00   -0.354993
2022-05-02 23:45:00   -0.354993
2022-05-03 00:00:00   -0.354993
Freq: 15T, Name: pv_measurement, Length: 116933, dtype: float64

'BC'

'est'

2022-10-28 22:00:00   -0.354993
2022-10-28 22:15:00   -0.354993
2022-10-28 22:30:00   -0.354993
2022-10-28 22:45:00   -0.354993
2022-10-28 23:00:00   -0.354993
                         ...   
2023-04-30 22:45:00   -0.354993
2023-04-30 23:00:00   -0.354993
2023-04-30 23:15:00   -0.354993
2023-04-30 23:30:00   -0.354993
2023-04-30 23:45:00   -0.354993
Freq: 15T, Name: pv_measurement, Length: 17672, dtype: float64

Do PCA of Xs with sklearn

In [28]:
from sklearn.decomposition import PCA

X_norm = pd.concat([Xs_norm[loc][type] for loc in Xs_norm for type in Xs_norm[loc]])

variance_threshold = 0.99
pca = PCA()
pca.fit(X_norm)

explained_variance = pca.explained_variance_ratio_
cumulative_variance = explained_variance.cumsum()
n_components = (cumulative_variance < variance_threshold).sum() + 1
display(n_components, X_norm.shape[1])

Xs_pca = {loc : {} for loc in Xs}
for loc in Xs:
    for type in Xs[loc]:
        Xs_pca[loc][type] = pd.DataFrame(
            scaler_X.transform(Xs[loc][type].to_numpy())[:, :n_components], 
            columns = [f'PC{i+1}' for i in range(n_components)],
            index = Xs[loc][type].index
        )

if display_plots:
    plt.plot(cumulative_variance, color="black")
    plt.xlabel("Number of Components")
    plt.ylabel("Cumulative Explained Variance")
    plt.axhline(0.99, linestyle='--', label='99% variance', color=(0.5, 0.0, 0.0))
    plt.axhline(0.95, linestyle='--', label='95% variance', color=(0.0, 0.5, 0.0))
    plt.axhline(0.90, linestyle='--', label='90% variance', color=(0.0, 0.0, 0.5))
    plt.legend()
    plt.show()

for loc in Xs:
    for type in Xs[loc]:
        display(loc, type, Xs_pca[loc][type])

32

45

'A'

'obs'

,PC1,PC2,PC3,PC4,PC5,PC6,PC7,PC8,PC9,PC10,PC11,PC12,PC13,PC14,PC15,PC16,PC17,PC18,PC19,PC20,PC21,PC22,PC23,PC24,PC25,PC26,PC27,PC28,PC29,PC30,PC31,PC32
2019-06-02 22:00:00,0.612617,-0.695508,-0.519220,-0.623954,-0.622388,-0.076851,-0.014095,0.734235,-0.643929,-0.652121,-0.439971,-0.446065,0.924236,-0.974891,-0.173847,-0.103819,-0.214494,-0.124194,-0.14371,-0.954833,0.867405,-0.233638,-0.176078,-0.218363,-0.131859,-0.133820,-0.168573,-0.223429,-0.032479,-0.145523,-0.183148,-0.005623
2019-06-02 22:15:00,0.612617,-0.722430,-0.523227,-0.623954,-0.622388,-0.082433,-0.014095,0.734235,-0.643929,-0.652121,-0.439971,-0.446065,0.932892,-0.974891,-0.173847,-0.103819,-0.214494,-0.124194,-0.14371,-0.954833,0.867405,-0.256725,-0.176078,-0.218363,-0.155108,-0.164684,-0.168573,-0.223429,-0.095389,-0.168570,-0.183148,-0.005623
2019-06-02 22:30:00,0.612617,-0.749349,-0.527086,-0.623954,-0.622388,-0.087809,-0.014095,0.734235,-0.643929,-0.652121,-0.439971,-0.446065,0.938662,-0.974891,-0.173847,-0.103819,-0.214494,-0.124194,-0.14371,-0.954833,0.867405,-0.287511,-0.176078,-0.218363,-0.178361,-0.187839,-0.168573,-0.223429,-0.165289,-0.199303,-0.183148,-0.005623
2019-06-02 22:45:00,0.612617,-0.803193,-0.530799,-0.623954,-0.622388,-0.092981,-0.014095,0.734235,-0.643929,-0.652121,-0.439971,-0.446065,0.944432,-0.974891,-0.173847,-0.103819,-0.214494,-0.124194,-0.14371,-0.954833,0.867405,-0.310598,-0.176078,-0.218363,-0.201610,-0.210988,-0.168573,-0.223429,-0.235189,-0.222350,-0.183148,-0.005623
2019-06-02 23:00:00,0.612617,-0.830112,-0.534401,-0.623954,-0.622388,-0.097999,-0.014095,0.734235,-0.643929,-0.652121,-0.439971,-0.446065,0.950203,-0.974891,-0.173847,-0.103819,-0.214494,-0.124194,-0.14371,-0.954833,0.867405,-0.333685,-0.176078,-0.218363,-0.224859,-0.234138,-0.168573,-0.223429,-0.305089,-0.245397,-0.183148,-0.005623
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-10-21 00:00:00,-0.366921,0.973592,-0.685885,-0.623954,-0.622388,-0.893764,-0.014095,-0.210112,-0.643929,-0.652121,-0.439971,-0.446065,-1.842610,-0.974891,-0.173847,-0.103819,-0.214494,-0.124194,-0.14371,-0.954833,0.867405,0.659097,-0.176078,-0.218363,0.743876,0.753618,-0.168573,-0.223429,0.638560,0.761036,-0.183148,-0.005623
2022-10-21 00:15:00,-0.366921,0.973592,-0.685885,-0.623954,-0.622388,-0.893764,-0.014095,-0.210112,-0.643929,-0.652121,-0.439971,-0.446065,-1.857036,-0.974891,-0.173847,-0.103819,-0.214494,-0.124194,-0.14371,-0.954833,0.867405,0.651398,-0.176078,-0.218363,0.736128,0.745903,-0.168573,-0.223429,0.666520,0.753355,-0.183148,-0.005623
2022-10-21 00:30:00,-0.366921,0.973592,-0.685885,-0.623954,-0.622388,-0.893764,-0.014095,-0.210112,-0.643929,-0.652121,-0.439971,-0.446065,-1.874347,-0.974891,-0.173847,-0.103819,-0.214494,-0.124194,-0.14371,-0.954833,0.867405,0.643704,-0.176078,-0.218363,0.728375,0.738183,-0.168573,-0.223429,0.687490,0.745674,-0.183148,-0.005623
2022-10-21 00:45:00,-0.366921,0.973592,-0.685885,-0.623954,-0.622388,-0.893764,-0.014095,-0.224422,-0.643929,-0.652121,-0.439971,-0.446065,-1.891658,-0.974891,-0.173847,-0.103819,-0.214494,-0.124194,-0.14371,-0.954833,0.867405,0.628311,-0.176078,-0.218363,0.712875,0.722753,-0.168573,-0.223429,0.715450,0.730308,-0.183148,-0.005623


...

'est'

,PC1,PC2,PC3,PC4,PC5,PC6,PC7,PC8,PC9,PC10,PC11,PC12,PC13,PC14,PC15,PC16,PC17,PC18,PC19,PC20,PC21,PC22,PC23,PC24,PC25,PC26,PC27,PC28,PC29,PC30,PC31,PC32
2022-10-28 22:00:00,0.848432,-0.641667,-0.636352,-0.623954,-0.622388,-0.348771,4.048477,0.877319,-0.643929,-0.652121,-0.439971,-0.446065,0.961743,0.329628,-0.173847,-0.103819,-0.214494,-0.124194,-0.14371,-0.954833,0.867405,-0.672310,-0.176078,-0.218363,-0.674351,-0.681714,-0.168573,-0.223429,1.379499,-0.683312,-0.183148,-0.005623
2022-10-28 22:15:00,0.848432,-0.641667,-0.393284,-0.623954,-0.622388,-0.205220,4.048477,0.877319,-0.643929,-0.652121,-0.439971,-0.446065,0.961743,0.329628,-0.173847,-0.103819,-0.214494,-0.124194,-0.14371,-0.954833,0.867405,-0.699244,-0.176078,-0.218363,-0.697599,-0.704864,-0.168573,0.971893,1.369013,-0.714041,-0.183148,-0.005623
2022-10-28 22:30:00,0.848432,-0.655129,-0.150234,-0.623954,-0.622388,-0.061644,4.048477,0.877319,-0.643929,-0.652121,-0.439971,-0.446065,0.961743,0.329628,-0.173847,-0.103819,-0.214494,-0.124194,-0.14371,-0.954833,0.867405,-0.722331,-0.176078,-0.218363,-0.720853,-0.728018,-0.168573,2.167216,1.355034,-0.737092,-0.183148,-0.005623
2022-10-28 22:45:00,0.830292,-0.668589,0.092835,-0.623954,-0.622388,0.081907,4.048477,0.877319,-0.643929,-0.652121,-0.439971,-0.446065,0.961743,0.329628,-0.173847,-0.103819,-0.214494,-0.124194,-0.14371,-0.954833,0.867405,-0.745418,-0.176078,-0.218363,-0.744101,-0.751168,-0.168573,2.167216,1.344549,-0.760140,-0.183148,-0.005623
2022-10-28 23:00:00,0.848432,-0.682048,0.335903,-0.623954,-0.622388,0.225484,4.048477,0.877319,-0.643929,-0.652121,-0.439971,-0.446065,0.961743,0.329628,-0.173847,-0.103819,-0.214494,-0.124194,-0.14371,-0.954833,0.867405,-0.772352,-0.176078,-0.218363,-0.771224,-0.778175,-0.168573,2.167216,1.334064,-0.783187,-0.183148,-0.005623
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-04-30 22:45:00,-0.566457,0.650538,-0.754440,-0.623954,-0.622388,-0.689049,-0.014095,-0.439047,-0.643929,-0.652121,-0.439971,-0.446065,0.953088,0.329628,-0.173847,-0.103819,-0.063321,-0.124194,-0.14371,-0.954833,0.867405,0.351258,-0.176078,-0.218363,0.302132,0.313757,-0.168573,-0.223429,0.435850,0.319280,-0.183148,-0.005623
2023-04-30 23:00:00,-0.566457,0.663998,-0.801676,-0.623954,-0.622388,-0.685721,-0.014095,-0.439047,-0.643929,-0.652121,-0.439971,-0.446065,0.961743,0.329628,-0.173847,-0.103819,-0.214494,-0.124194,-0.14371,-0.954833,0.867405,0.347406,-0.176078,-0.218363,0.298258,0.306042,-0.168573,-0.223429,0.463810,0.315440,-0.183148,-0.005623
2023-04-30 23:15:00,-0.566457,0.650538,-0.659602,-0.623954,-0.622388,-0.690278,-0.014095,-0.439047,-0.643929,-0.652121,-0.439971,-0.446065,0.948760,0.329628,-0.173847,-0.103819,-0.063321,-0.124194,-0.14371,-0.954833,0.867405,0.335865,-0.176078,-0.218363,0.286631,0.298327,-0.168573,-0.223429,0.449830,0.300073,-0.183148,-0.005623
2023-04-30 23:30:00,-0.566457,0.650538,-0.517529,-0.623954,-0.622388,-0.694835,-0.014095,-0.439051,-0.643929,-0.652121,-0.439971,-0.446065,0.919909,0.329628,-0.173847,-0.103819,-0.063321,-0.124194,-0.14371,-0.954833,0.867405,0.324319,-0.176078,-0.218363,0.278883,0.282892,-0.168573,-0.223429,0.435850,0.292393,-0.183148,-0.005623


'BC'

'test'

,PC1,PC2,PC3,PC4,PC5,PC6,PC7,PC8,PC9,PC10,PC11,PC12,PC13,PC14,PC15,PC16,PC17,PC18,PC19,PC20,PC21,PC22,PC23,PC24,PC25,PC26,PC27,PC28,PC29,PC30,PC31,PC32
2023-05-01 00:00:00,-0.639015,0.771683,-0.825275,-0.623954,-0.622388,-0.402485,-0.014095,-0.539203,-0.643929,-0.652121,-0.439971,-0.446065,0.303932,0.329628,-0.173847,-0.103819,-0.214494,-0.124194,-0.14371,-0.954833,0.867405,0.320472,-0.176078,-0.218363,0.271135,0.282892,-0.168573,-0.223429,0.498760,0.292393,-0.183148,-0.005623
2023-05-01 00:15:00,-0.639015,0.771683,-0.825275,-0.623954,-0.622388,-0.389120,-0.014095,-0.539203,-0.643929,-0.652121,-0.439971,-0.446065,0.210165,0.329628,-0.173847,-0.103819,-0.214494,-0.124194,-0.14371,-0.954833,0.867405,0.305079,-0.176078,-0.218363,0.263383,0.267457,-0.168573,-0.223429,0.495265,0.277026,-0.183148,-0.005623
2023-05-01 00:30:00,-0.657154,0.771683,-0.825275,-0.623954,-0.622388,-0.375730,-0.014095,-0.553516,-0.643929,-0.652121,-0.439971,-0.446065,0.112070,0.329628,-0.173847,-0.103819,-0.214494,-0.124194,-0.14371,-0.954833,0.867405,0.297385,-0.176078,-0.218363,0.255635,0.259742,-0.168573,-0.223429,0.488275,0.269345,-0.183148,-0.005623
2023-05-01 00:45:00,-0.657154,0.771683,-0.825275,-0.623954,-0.622388,-0.362366,-0.014095,-0.560669,-0.643929,-0.652121,-0.439971,-0.446065,0.006763,0.329628,-0.173847,-0.103819,-0.214494,-0.124194,-0.14371,-0.954833,0.867405,0.289687,-0.176078,-0.218363,0.240134,0.252023,-0.168573,-0.223429,0.484780,0.261660,-0.183148,-0.005623
2023-05-01 01:00:00,-0.657154,0.771683,-0.720944,-0.623954,-0.622388,-0.348951,-0.014095,-0.560669,-0.643929,-0.652121,-0.439971,-0.446065,-0.102872,0.329628,-0.173847,-0.103819,-0.214494,-0.124194,-0.14371,-0.954833,0.867405,0.278145,-0.176078,-0.218363,0.232386,0.244308,-0.168573,-0.223429,0.477790,0.253979,-0.183148,-0.005623
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-07-03 22:45:00,1.084247,-1.328149,-0.408079,-0.623954,-0.622388,-0.150150,-0.014095,1.070487,-0.643929,-0.652121,-0.439971,-0.446065,0.951645,0.329628,-0.173847,-0.103819,-0.214494,-0.124194,-0.14371,-0.954833,0.867405,-1.264896,-0.176078,-0.218363,-1.251717,-1.264335,-0.168573,-0.223429,0.663025,-1.278722,-0.183148,-0.005623
2023-07-03 23:00:00,1.102386,-1.314690,-0.421607,-0.623954,-0.622388,-0.199793,-0.014095,1.063330,-0.643929,-0.652121,-0.439971,-0.446065,0.961743,0.329628,-0.173847,-0.103819,-0.214494,-0.124194,-0.14371,-0.954833,0.867405,-1.261048,-0.176078,-0.218363,-1.247843,-1.260478,-0.168573,-0.223429,0.677005,-1.271037,-0.183148,-0.005623
2023-07-03 23:15:00,1.084247,-1.314690,-0.421643,-0.623954,-0.622388,-0.248206,-0.014095,1.063330,-0.643929,-0.652121,-0.439971,-0.446065,0.961743,0.329628,-0.173847,-0.103819,-0.214494,-0.124194,-0.14371,-0.954833,0.867405,-1.257201,-0.176078,-0.218363,-1.243969,-1.256620,-0.168573,-0.223429,0.701470,-1.267197,-0.183148,-0.005623
2023-07-03 23:30:00,1.066107,-1.314690,-0.421680,-0.623954,-0.622388,-0.296620,-0.014095,1.063330,-0.643929,-0.652121,-0.439971,-0.446065,0.961743,0.329628,-0.173847,-0.103819,-0.214494,-0.124194,-0.14371,-0.954833,0.867405,-1.249507,-0.176078,-0.218363,-1.236217,-1.248905,-0.168573,-0.223429,0.725934,-1.263356,-0.183148,-0.005623


# Darts

In [29]:
from darts import TimeSeries

future_cov = {loc: {type: TimeSeries.from_dataframe(Xs[loc][type]) for type in Xs[loc]} for loc in Xs}
target = {loc: {type: TimeSeries.from_series(ys[loc][type]) for type in ys[loc]} for loc in ys}

future_cov_norm = {loc: {type: TimeSeries.from_dataframe(Xs_norm[loc][type]) for type in Xs_norm[loc]} for loc in Xs_norm}
target_norm = {loc: {type: TimeSeries.from_series(ys_norm[loc][type]) for type in ys_norm[loc]} for loc in ys_norm}

future_cov_pca = {loc: {type: TimeSeries.from_dataframe(Xs_pca[loc][type]) for type in Xs_pca[loc]} for loc in Xs_pca}